In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PP.jl")
include("Estimation_PP.jl")
include("Evaluation_PP.jl")
# # include("Plot_Figures.jl")
include("Implement_All_Methods_PP.jl")
include("Models_PP.jl")

Solve_RO (generic function with 1 method)

## Functions

In [2]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, gamma_list, Time_Limit,dual_norm,seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    Params["offdiag_sign"] = offdiag_sign
    Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    Params["gamma_list"] = gamma_list
    Params["Time_Limit"] = Time_Limit
    Params["dual_norm"] = dual_norm
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

In [3]:
function Generate_Input_this(S_train, N, N_u, K, offdiag_sign,max_offdiag,P_bar,is_original_setting)
    Input_Data_this = Dict()
    A_true, B_true = Generate_Coef(N_u, N, max_offdiag, offdiag_sign,is_original_setting);
    P_train,PM_train,choice_train,PM_train_extend = Generate_Data(N,S_train,A_true,B_true,P_bar);

    Input_Data_this["A_true"] = A_true;
    Input_Data_this["B_true"] = B_true;
    Input_Data_this["P_dag"] = round.(rand(N, K) .* P_bar; digits=2);
    Input_Data_this["P_train"] = P_train;
    Input_Data_this["PM_train_extend"] = PM_train_extend;
    Input_Data_this["choice_train"] = choice_train;

    A_hat,B_hat = Estimate_MNL_Para(PM_train_extend, P_train, choice_train,S_train, N);

    Input_Data_this["A_hat"] = A_hat
    Input_Data_this["B_hat"] = B_hat
    return Input_Data_this
end

Generate_Input_this (generic function with 1 method)

In [4]:
function Get_Input_Data(Input_Data_this)
    A_true = Input_Data_this["A_true"]
    B_true = Input_Data_this["B_true"]
    P_dag = Input_Data_this["P_dag"]
    P_train = Input_Data_this["P_train"]
    PM_train_extend = Input_Data_this["PM_train_extend"]
    choice_train = Input_Data_this["choice_train"]
    A_hat = Input_Data_this["A_hat"]
    B_hat = Input_Data_this["B_hat"]
    return A_true,B_true,P_dag,P_train,PM_train_extend,choice_train,A_hat,B_hat
end

Get_Input_Data (generic function with 1 method)

In [5]:
function solve_ETO_this(N,N_u,K,A,B,A_true, B_true,P_dag,Time_Limit)
    RST_this = Dict()
    status_this = "NotDefined"
    obj_,X_,Promo_, time_,status_ = Solve_ETO(N,N_u,K,A, B,P_dag,Time_Limit)
    if status_ != "OPTIMAL"
        status_this = status_
    else
        status_this = status_
        rev_, price_ = compute_oof(X_, A_true, B_true, vcat(Promo_,1), P_dag)
        RST_this["obj"] = obj_
        RST_this["price"] = price_
        RST_this["promo"] = Promo_
        RST_this["time"] = time_
        RST_this["Rev"] = rev_
        RST_this["status"] = status_
    end
    return RST_this,status_this
end

solve_ETO_this (generic function with 1 method)

In [6]:
function solve_RO_this(N,N_u,K,A_hat,B_hat,A_true, B_true,P_dag,psi_lb,psi_ub,phi_lb,phi_ub,gamma_list,dual_norm,Time_Limit)
    RST_this = Dict()
    status_this = "NotDefined"
    for gamma in gamma_list
        obj_RO,X_RO,Promo_RO, time_RO,status_RO = Solve_RO(N,N_u,K,A_hat,B_hat,P_dag,psi_lb,psi_ub,phi_lb,phi_ub,gamma * ones(N),dual_norm,Time_Limit)
        println("gamma=$(gamma),status:",status_RO,",time=$time_RO")
        if status_RO != "OPTIMAL"
            status_this = status_RO
            break
        else
            status_this = status_RO
            rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, vcat(Promo_RO,1), P_dag)
            RST_this["obj_gamma=$(gamma)"] = obj_RO
            RST_this["price_gamma=$(gamma)"] = price_RO
            RST_this["promo_gamma=$(gamma)"] = Promo_RO
            RST_this["time_gamma=$(gamma)"] = time_RO
            RST_this["Rev_gamma=$(gamma)"] = rev_RO
            RST_this["status"] = status_this
        end
    end
    return RST_this,status_this
end

solve_RO_this (generic function with 1 method)

#### Parameters

In [7]:
seed = 1;
N = 3;
N_u = 1;
K = 30;
S_train = 50;
S_test = 1000;
P_bar = 30.0;
iterations = 100;
project_dir = "Joint_Pricing_and_Promotion/"

offdiag_sign = "mix";
max_offdiag = 1.0;
Time_Limit = 180.0
gamma_list = [0.0,0.01,0.02,0.03,0.04,0.06,0.08];
dual_norm = 2

psi_lb_coef = -20.0
phi_lb_coef = -20.0
psi_lb = psi_lb_coef * ones(N) 
psi_ub = 0.0 * ones(N) 
phi_lb = phi_lb_coef * ones(N) 
phi_ub = 0.0 * ones(N) 

is_original_setting = true
Random.seed!(seed)

TaskLocalRNG()

In [8]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/")
if !isdir(data_dir)
    mkpath(data_dir)
end
if is_original_setting
    sub_file_name = "MS_2024_N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_Seed=$seed/"
else
    sub_file_name = "N=$(N)_N_u=$(N_u)_K=$(K)_S_train=$(S_train)_offdiag_sign=$(offdiag_sign)_max_offdiag=$(max_offdiag)/"
end
this_data_file = string(data_dir,project_dir,sub_file_name)
if !isdir(this_data_file)
    mkpath(this_data_file)
end

In [9]:
Params = set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, gamma_list, Time_Limit,dual_norm,seed);
save(string(this_data_file, "Params.jld2"), Params);
println(this_data_file)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Joint_Pricing_and_Promotion/MS_2024_N=3_N_u=1_K=30_S_train=50_Seed=1/


In [ ]:
Result_All = Dict()
iter = 1
while iter <= iterations
    Input_Data_this = Generate_Input_this(S_train, N, N_u, K, offdiag_sign,max_offdiag,P_bar,is_original_setting)
    A_true,B_true,P_dag,P_train,PM_train_extend,choice_train,A_hat,B_hat = Get_Input_Data(Input_Data_this)
    if any(isnan, A_hat) || any(isnan, B_hat)
        println("Estimate contains NaN values")
        continue
    end
    Result_All["Input_Data_iter=$(iter)"] = Input_Data_this

    RST_Oracle,status_Oracle = solve_ETO_this(N,N_u,K,A_true,B_true,A_true, B_true,P_dag,Time_Limit)
    println("Oracle: status = ",status_Oracle,",time=",RST_Oracle["time"])
    if status_Oracle != "OPTIMAL"
        println("Oracle did not approach the optimal solution")
        continue
    end
    Result_All["RST_Oracle_iter=$(iter)"] = RST_Oracle

    RST_ETO,status_ETO = solve_ETO_this(N,N_u,K,A_hat,B_hat,A_true, B_true,P_dag,Time_Limit)
    println("ETO: status = ",status_ETO,",time=",RST_ETO["time"])
    if status_ETO != "OPTIMAL"
        println("ETO did not approach the optimal solution")
        continue
    end
    Result_All["RST_ETO_iter=$(iter)"] = RST_ETO
    
    obj_RO,X_RO,Promo_RO, time_RO,status_RO = Solve_RO(N,N_u,K,A_hat,B_hat,P_dag,psi_lb,psi_ub,phi_lb,phi_ub,0.0 * ones(N),dual_norm,Time_Limit)
    if status_RO != "OPTIMAL"
        println("RO did not approach the optimal solution")
        continue
    end
    rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, vcat(Promo_RO,1), P_dag)
    if abs(rev_RO - RST_ETO["Rev"]) >= 0.001
        println("ETO rev is not equivalent to RO rev")
        continue
    end

    RST_RO,status_RO = solve_RO_this(N,N_u,K,A_hat,B_hat,A_true, B_true,P_dag,psi_lb,psi_ub,phi_lb,phi_ub,gamma_list,dual_norm,Time_Limit)
    # println("RO status = ",status_RO)
    if status_RO != "OPTIMAL"
        println("RO hello world")
        continue
    end
    # Result_All["Input_Data_iter=$(iter)"] = Input_Data_this
    # Result_All["RST_Oracle_iter=$(iter)"] = RST_Oracle
    # Result_All["RST_ETO_iter=$(iter)"] = RST_ETO
    Result_All["RST_RO_iter=$(iter)"] = RST_RO
    save(string(this_data_file, "Result_All.jld2"), Result_All);
    println("******* iter = ",iter,"*********")
    iter = iter + 1
end
save(string(this_data_file, "Result_All.jld2"), Result_All);

Oracle: status = OPTIMAL,time=1.7175960540771484
ETO: status = OPTIMAL,time=2.031052827835083
gamma=0.0,status:OPTIMAL,time=38.481637954711914
gamma=0.01,status:OPTIMAL,time=29.32484197616577
gamma=0.02,status:OPTIMAL,time=25.874020099639893
gamma=0.03,status:OPTIMAL,time=26.943052053451538
gamma=0.04,status:OPTIMAL,time=36.25081992149353
gamma=0.06,status:OPTIMAL,time=32.40744614601135
gamma=0.08,status:OPTIMAL,time=21.850903034210205
******* iter = 1*********
Oracle: status = OPTIMAL,time=1.3869779109954834
ETO: status = OPTIMAL,time=5.589294910430908
gamma=0.0,status:OPTIMAL,time=44.13572812080383
gamma=0.01,status:OPTIMAL,time=29.152417182922363
gamma=0.02,status:OPTIMAL,time=24.200669050216675
gamma=0.03,status:OPTIMAL,time=17.91047477722168
gamma=0.04,status:OPTIMAL,time=14.023545980453491
gamma=0.06,status:OPTIMAL,time=10.964210033416748
gamma=0.08,status:OPTIMAL,time=12.965451002120972
******* iter = 2*********
Oracle: status = OPTIMAL,time=1.3673739433288574
ETO: status = OPTI

In [ ]:
Rev_Oracle = [Result_All["RST_Oracle_iter=$(iter)"]["Rev"] for iter in 1:iterations]
Rev_ETO = [Result_All["RST_ETO_iter=$(iter)"]["Rev"] for iter in 1:iterations]
Rev_RO = zeros(iterations,length(gamma_list))
for iter in 1:iterations
    RST_RO = Result_All["RST_RO_iter=$(iter)"]
    for g_index in 1:length(gamma_list)
        gamma = gamma_list[g_index]
        Rev_RO[iter,g_index] = RST_RO["Rev_gamma=$(gamma)"]
    end
end
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))
println("Rev_RO/Rev_Oracle = ",round.(Rev_RO[:,1]./Rev_Oracle,digits=4))
println("Rev_ETO/Rev_Oracle = ",mean(Rev_ETO)/mean(Rev_Oracle))

In [ ]:
RO_ETO_Ratio_Mean = zeros(length(gamma_list))
RO_ETO_Ratio_Std = zeros(length(gamma_list))
for g_index in 1:length(gamma_list)
    gamma = gamma_list[g_index]
    RO_ETO_Ratio_Mean[g_index] = mean(Rev_RO[:,g_index])/mean(Rev_ETO)
    RO_ETO_Ratio_Std[g_index] = std(Rev_RO[:,g_index])/std(Rev_ETO)
    println("gamma=$gamma,Mean =",RO_ETO_Ratio_Mean[g_index],", Std =",RO_ETO_Ratio_Std[g_index])
end

In [ ]:
end_index = length(gamma_list)
plot(gamma_list[1:end_index], RO_ETO_Ratio_Mean[1:end_index], marker=:o, xlabel=L"\gamma", label="Average Profit", xticks=(gamma_list[1:end_index], string.(gamma_list[1:end_index])))
plot!(gamma_list[1:end_index], RO_ETO_Ratio_Std[1:end_index], marker=:o, xlabel=L"\gamma", ylabel="RPD/ETOPD", label="Std of Profit")
hline!([1.0], linestyle=:dash, color=:red, label="")
savefig(string(this_data_file, "RPP_vs_ETOPP.pdf"))

In [ ]:
bad_idx = findall(i -> abs(Rev_ETO[i] - Rev_RO[i,1]) > 0.01, 1:length(Rev_ETO));
keep_idx = setdiff(1:length(Rev_ETO), bad_idx);
Rev_ETO_clean = Rev_ETO[keep_idx];
Rev_Oracle_clean = Rev_Oracle[keep_idx];
Rev_RO_clean = Rev_RO[keep_idx, :];
RO_ETO_Ratio_Mean = zeros(length(gamma_list))
RO_ETO_Ratio_Std = zeros(length(gamma_list))
for g_index in 1:length(gamma_list)
    gamma = gamma_list[g_index]
    RO_ETO_Ratio_Mean[g_index] = mean(Rev_RO_clean[:,g_index])/mean(Rev_ETO_clean)
    RO_ETO_Ratio_Std[g_index] = std(Rev_RO_clean[:,g_index])/std(Rev_ETO_clean)
    println("gamma=$gamma,Mean =",RO_ETO_Ratio_Mean[g_index],", Std =",RO_ETO_Ratio_Std[g_index])
end
println("Rev_ETO/Rev_Oracle = ",mean(Rev_ETO_clean)/mean(Rev_Oracle_clean))

In [ ]:
end_index = length(gamma_list) -1
plot(gamma_list[1:end_index], RO_ETO_Ratio_Mean[1:end_index], marker=:o, xlabel=L"\gamma", label="Average Profit", xticks=(gamma_list[1:end_index], string.(gamma_list[1:end_index])))
plot!(gamma_list[1:end_index], RO_ETO_Ratio_Std[1:end_index], marker=:o, xlabel=L"\gamma", ylabel="RPD/ETOPD", label="Std of Profit")
hline!([1.0], linestyle=:dash, color=:red, label="")
savefig(string(this_data_file, "RPP_vs_ETOPP_Clean.pdf"))